In [2]:
import pandas as pd
import numpy as np

In [33]:
haber = pd.read_csv("../Data/banks.csv")


In [34]:
haber.head()

,Unnamed: 0,title,content,date,bank
0,0,Endekste 101.000 sağlam destek,"""BIST 100 Endeksi, dünü % 0.23 artışla, 102,43...",15 Mart 2019,akbank
1,1,Bankalar uçarken havayolları iniyor,"""Bankalar uçarken havayolları iniyor Dolar güç...",14 Mart 2019,akbank
2,2,Şirket haberleri ve tavsiyeleri 13/3/2019,"Akbank (AKBNK, Öneri ""EKLE"", Nötr): Banka ta...",13 Mart 2019,akbank
3,3,Şirket haberleri ve tavsiyeleri 12/3/2019,"Alarko Holding (ALARK, Sınırlı Pozitif): Alark...",12 Mart 2019,akbank
4,4,Hazine'nin yeni projesine 12 banka ortak oluyor,Hazine ve Maliye Bakanlığı 10 milyon TL sermay...,08 Mart 2019,akbank


In [35]:
months = ["ocak","şubat","mart","nisan","mayıs","haziran","temmuz","ağustos","eylül","ekim","kasım","aralık"]
months_dict = {months[i] : i+1 for i in range(0,12)}

In [36]:
for i in range(len(haber.date)):
    try:
        ay = str(haber.date[i].split()[1].lower())
        haber.date[i] = f"{haber.date[i].split()[0]}.{months_dict[ay]}.{haber.date[i].split()[2]}"
    except Exception as e:
        pass


C:\Users\Emirhan\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [283]:
haber.tail()

,title,content,date,bank
12341,Köymen'den farklı yorumlar!,Haute Couture’nin ustası; Türkiye’nin yetiştir...,2013-03-28,denizbank
12342,O'na ne önerdi?,Haute Couture’nin ustası; Türkiye’nin yetiştir...,2013-03-27,denizbank
12343,'Yüz hatları çok karizmatik',Haute Couture’nin ustası; özgür giyimin markas...,2013-03-26,denizbank
12344,Aracı kurumlar da yabancılaşıyor,Yabancı sermayenin Türkiye’ye olan ilgisi sade...,2013-03-05,denizbank
12345,Bankacıların iddialı 2013 hedefleri,Bütün bankacılar aslında hem fikir. Hepsine gö...,2013-01-09,denizbank


In [38]:
haber.date = pd.to_datetime(haber["date"]).dt.strftime('%d.%m.%Y')


In [40]:
haber.to_csv("banks.csv",index=False)

In [45]:
haber.drop("Unnamed: 0",axis=1,inplace=True)

In [47]:
haber.tail()

,title,content,date,bank
12390,Köymen'den farklı yorumlar!,Haute Couture’nin ustası; Türkiye’nin yetiştir...,28.03.2013,denizbank
12391,O'na ne önerdi?,Haute Couture’nin ustası; Türkiye’nin yetiştir...,27.03.2013,denizbank
12392,'Yüz hatları çok karizmatik',Haute Couture’nin ustası; özgür giyimin markas...,26.03.2013,denizbank
12393,Aracı kurumlar da yabancılaşıyor,Yabancı sermayenin Türkiye’ye olan ilgisi sade...,03.05.2013,denizbank
12394,Bankacıların iddialı 2013 hedefleri,Bütün bankacılar aslında hem fikir. Hepsine gö...,01.09.2013,denizbank


In [51]:
haber.dropna(inplace=True)

In [49]:
haber.dropna()

title      29
content    20
date        0
bank        0
dtype: int64

In [61]:
haber.reset_index(drop=True,inplace=True)

In [66]:
haber.drop("index",axis=1,inplace=True)

In [68]:
haber.to_csv("banks.csv",index=False)

## Hisse Oranları

In [72]:
import os
os.listdir("../Data/hisseler")

['ak.csv',
 'deniz.csv',
 'garanti.csv',
 'halk.csv',
 'is.csv',
 'vakif.csv',
 'yapi.csv']

In [123]:
banks = ["akbank","denizbank","garanti","halkbank","is%20bankasi","vakifbank","yapi%20kredi"]
banks_dict = {}
path = "../Data/hisseler/"
index = 0
for i in os.listdir("../Data/hisseler")[1:]:
    banks_dict[banks[index]] =  pd.read_csv(path+i)
    banks_dict[banks[index]].rename(columns={"Tarih":"date"},inplace=True)
    banks_dict[banks[index]].date = pd.to_datetime(banks_dict[banks[index]]["date"])
    banks_dict[banks[index]].set_index("date",inplace=True)
    index +=1

In [124]:
haber.date = pd.to_datetime(haber["date"])

## Günleri Eşleştirme

In [284]:

hs = {}
for b in banks: 
    liste = []
    liste_date = []
    a  = 0
    for i in haber[haber.bank==b].date:
        tarih = i + pd.Timedelta(days=1)  
        try:

            liste.append(banks_dict[b].loc[tarih,col].values)
            liste_date.append(tarih)
        except:
            try:
                liste.append(banks_dict[b].loc[tarih + pd.Timedelta(days=1),col].values)
                liste_date.append(tarih)
            except:
                try:
                    liste.append(banks_dict[b].loc[tarih + pd.Timedelta(days=2),col].values)
                    liste_date.append(tarih)
                except:
                    liste.append([np.nan,np.nan,np.nan,np.nan])
                    liste_date.append(np.nan)
                    a +=1
    #                 try:
    #                     liste.append(banks_dict["akbank"].loc[tarih + pd.Timedelta(days=3),col].values)
    #                 except:
    #                     try:
    #                         liste.append(banks_dict["akbank"].loc[tarih + pd.Timedelta(days=4),col].values)
    #                     except:
    #                         liste.append(np.nan)
    hs[b] = [liste,liste_date]
    print(a)

21
529
54
27
39
55
44


In [244]:
len(haber)

12346

## Haber ve Günleri Eşleştirme

In [403]:
result = pd.DataFrame()
for b in banks:
    ba = haber[haber.bank==b].reset_index(drop=True)
    data = pd.DataFrame(data=hs[b][0],columns=col)
    tarih = pd.DataFrame(data=hs[b][1],columns=["next_date"])
    conct = pd.concat([ba,tarih,data],axis=1).reset_index(drop=True)
    result = pd.concat([result,conct],axis=0).reset_index(drop=True)

In [409]:
result.dropna(inplace=True)

In [410]:
len(result)

11577

In [419]:
result.to_csv("../Data/Labelsız eşleştirilmiş data.csv",index=False)

In [412]:
result.head()

,title,content,date,bank,next_date,Şimdi,Açılış,Yüksek,Fark %
0,Endekste 101.000 sağlam destek,"""BIST 100 Endeksi, dünü % 0.23 artışla, 102,43...",2019-03-15,akbank,2019-03-16,"6,88","6,73","6,92","2,84%"
1,Bankalar uçarken havayolları iniyor,"""Bankalar uçarken havayolları iniyor Dolar güç...",2019-03-14,akbank,2019-03-15,"6,69","6,60","6,72","1,83%"
2,Şirket haberleri ve tavsiyeleri 13/3/2019,"Akbank (AKBNK, Öneri ""EKLE"", Nötr): Banka ta...",2019-03-13,akbank,2019-03-14,"6,57","6,61","6,70","-0,61%"
3,Şirket haberleri ve tavsiyeleri 12/3/2019,"Alarko Holding (ALARK, Sınırlı Pozitif): Alark...",2019-03-12,akbank,2019-03-13,"6,61","6,45","6,66","2,32%"
4,Hazine'nin yeni projesine 12 banka ortak oluyor,Hazine ve Maliye Bakanlığı 10 milyon TL sermay...,2019-03-08,akbank,2019-03-09,"7,27","7,15","7,34","1,39%"


## Label

In [413]:
result["Fark %"]=result["Fark %"].str.replace("%","")
result["Fark %"]=result["Fark %"].str.replace(",",".")

In [414]:
result.head()

,title,content,date,bank,next_date,Şimdi,Açılış,Yüksek,Fark %
0,Endekste 101.000 sağlam destek,"""BIST 100 Endeksi, dünü % 0.23 artışla, 102,43...",2019-03-15,akbank,2019-03-16,"6,88","6,73","6,92",2.84
1,Bankalar uçarken havayolları iniyor,"""Bankalar uçarken havayolları iniyor Dolar güç...",2019-03-14,akbank,2019-03-15,"6,69","6,60","6,72",1.83
2,Şirket haberleri ve tavsiyeleri 13/3/2019,"Akbank (AKBNK, Öneri ""EKLE"", Nötr): Banka ta...",2019-03-13,akbank,2019-03-14,"6,57","6,61","6,70",-0.61
3,Şirket haberleri ve tavsiyeleri 12/3/2019,"Alarko Holding (ALARK, Sınırlı Pozitif): Alark...",2019-03-12,akbank,2019-03-13,"6,61","6,45","6,66",2.32
4,Hazine'nin yeni projesine 12 banka ortak oluyor,Hazine ve Maliye Bakanlığı 10 milyon TL sermay...,2019-03-08,akbank,2019-03-09,"7,27","7,15","7,34",1.39


In [416]:
result['Fark %'] = result['Fark %'].astype(float)
def classify(fark):
    if fark > 1:
        return 1
    elif fark < -1:
        return -1
    elif -1< fark < 1:
        return 0
result["label"] = result.apply(lambda x:classify(x["Fark %"]),axis=1)

In [417]:
result.head()

,title,content,date,bank,next_date,Şimdi,Açılış,Yüksek,Fark %,label
0,Endekste 101.000 sağlam destek,"""BIST 100 Endeksi, dünü % 0.23 artışla, 102,43...",2019-03-15,akbank,2019-03-16,"6,88","6,73","6,92",2.84,1.0
1,Bankalar uçarken havayolları iniyor,"""Bankalar uçarken havayolları iniyor Dolar güç...",2019-03-14,akbank,2019-03-15,"6,69","6,60","6,72",1.83,1.0
2,Şirket haberleri ve tavsiyeleri 13/3/2019,"Akbank (AKBNK, Öneri ""EKLE"", Nötr): Banka ta...",2019-03-13,akbank,2019-03-14,"6,57","6,61","6,70",-0.61,0.0
3,Şirket haberleri ve tavsiyeleri 12/3/2019,"Alarko Holding (ALARK, Sınırlı Pozitif): Alark...",2019-03-12,akbank,2019-03-13,"6,61","6,45","6,66",2.32,1.0
4,Hazine'nin yeni projesine 12 banka ortak oluyor,Hazine ve Maliye Bakanlığı 10 milyon TL sermay...,2019-03-08,akbank,2019-03-09,"7,27","7,15","7,34",1.39,1.0


In [422]:
result.drop(["title","date","next_date","Şimdi","Açılış","Yüksek","Fark %"],axis=1,inplace=True)

In [424]:
result.to_csv("../Data/Labelled_news.csv",index=False)

In [423]:
result

,content,bank,label
0,"""BIST 100 Endeksi, dünü % 0.23 artışla, 102,43...",akbank,1.0
1,"""Bankalar uçarken havayolları iniyor Dolar güç...",akbank,1.0
2,"Akbank (AKBNK, Öneri ""EKLE"", Nötr): Banka ta...",akbank,0.0
3,"Alarko Holding (ALARK, Sınırlı Pozitif): Alark...",akbank,1.0
4,Hazine ve Maliye Bakanlığı 10 milyon TL sermay...,akbank,1.0
...,...,...,...
12341,"Ziraat Yatırım, Yapı Kredi Bankası hedef fiyat...",yapi%20kredi,0.0
12342,"Ziraat Yatırım, ( www.ziraatyatirim.com.tr ) k...",yapi%20kredi,0.0
12343,"Türk banka hisselerinin dün yüzde 7,4 değer ka...",yapi%20kredi,0.0
12344,Bugünlerde tüm bankalar yeni yılı planlama tel...,yapi%20kredi,0.0
